In [172]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time
import random
import itertools

In [181]:
def login(USERNAME, PASSWORD, driver):
    # メアドとパスワードを入力
    driver.find_element_by_name('username').send_keys(USERNAME)
    time.sleep(1)
    driver.find_element_by_name('password').send_keys(PASSWORD)
    time.sleep(1)

    # ログインボタンを押す
    driver.find_element_by_class_name('L3NKy').click()
    time.sleep(random.randint(2, 5))
    
    
def tagsearch(tagurl, driver):
    # tagで渡ってきたハッシュタグのページにアクセス
    driver.get(tagurl)
    driver.implicitly_wait(30)

# urlが投稿ページかどうかを確認
def is_post_url(url):
    if "/p/" in url:
        return True
    else:
        return False
    
def is_liked(soup):
    if soup.find("svg", {"aria-label": "「いいね！」を取り消す"}) is None:
        return True
    else:
        return False

In [182]:
def make_posturllist(tag, driver): # tagのpostのURLlsitを作成する
    posturllist = []
    # aタグを取得
    itemlist = driver.find_elements_by_tag_name("a")
    urllist = [item.get_attribute("href") for item in itemlist]
    posturllist = [s for s in urllist if "/p/" in s]
    time.sleep(1)
        
    return posturllist

In [183]:
def reload_makepostlist(driver, tagurl, posturllist, NEWPOST_COUNT):
    # 最新の投稿が見える位置まで移動
    target = driver.find_elements_by_class_name('_aabd')[len(posturllist)-NEWPOST_COUNT]
    actions = ActionChains(driver)
    actions.move_to_element(target)
    actions.perform()
    driver.implicitly_wait(30)
    time.sleep(1)
    
    #URLを取り直す
    itemlist = driver.find_elements_by_tag_name("a")
    urlList = [item.get_attribute("href") for item in itemlist]
    temp = [s for s in urlList if "/p/" in s]
    new_posturllist = posturllist + temp
    new_posturllist = list(set(new_posturllist))
    time.sleep(1)
    
    return new_posturllist

In [184]:
def pushnice(posturllist, NICE_MAX, driver, NEWPOST_COUNT): # いいねの回数を引数で指定
    nice_count = 0
    
    #人気の投稿を消して、最新の投稿だけに絞る
    new_posturllist = posturllist[NEWPOST_COUNT:]
    
    for url in new_posturllist:
        if nice_count < NICE_MAX:
            driver.get(url)
            driver.implicitly_wait(30)

            # htmlの取得
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # いいね！が押されていなかったら押す
            if is_liked(soup):
                driver.find_element_by_css_selector("section span button div span svg[aria-label=いいね！").click()

            # いいね！を押した回数をカウント
            nice_count += 1

            #数秒待機
            time.sleep(random.choice(range(1, 6)))
        else:
            break

In [185]:
# いいねの動きに対するパラメータ
USERNAME = ''
PASSWORD = ''

# いいねの最大回数
NICE_MAX = 3

#最初の9投稿は最新のものではない
NEWPOST_COUNT = 10

# 検索するハッシュタグを指定する。
taglist = ['camera']

In [186]:
if __name__ == '__main__':
    # Chrome Driverでインスタグラムを起動
    driver = webdriver.Chrome(executable_path='/opt/homebrew/bin/chromedriver')
    driver.implicitly_wait(30)

    driver.get('https://www.instagram.com/accounts/login/?source=auth_switcher')
    time.sleep(1)

    login(USERNAME, PASSWORD, driver)
    
    for tag in taglist:
        tagurl = 'https://www.instagram.com/explore/tags/' + tag
        # ハッシュタグで検索
        tagsearch(tagurl, driver)
        #
        posturllist = make_posturllist(tag, driver)

        while NICE_MAX > (len(posturllist) - 20):
            # 集めた投稿の数がNICE_MAXを越えるまで投稿のURLを集め続ける
            posturllist = reload_makepostlist(driver, tagurl, posturllist, NEWPOST_COUNT)

        # 集めた投稿全てにNICE_MAX分のいいねを押す
        pushnice(posturllist, NICE_MAX, driver, NEWPOST_COUNT)